In [1]:
import numpy as np
import scipy.io
from itertools import combinations
from scipy.special import comb
import pickle as pkl
import os

In [2]:
def import_data(patient, d=3):
    '''...'''
    data = scipy.io.loadmat(f'/Users/anibal/Google Drive/Information cochains/O-info/OinfoSinfo/OinfoCopulasN{d+1}/Oinfop{patient}.mat')

    del data['__header__']
    del data['__version__']
    del data['__globals__']

    data['TC']  = (data['Sinfo'] + data['Oinfo'])/2
    data['DTC'] = (data['Sinfo'] - data['Oinfo'])/2
    
    return data

In [3]:
def construct_boundary(d, r=19):
    '''...'''
    domain_basis = tuple(combinations(range(r+1), d+1))
    target_basis = tuple(combinations(range(r+1), d))
    target_basis_ix = {tuple(v): index for index, v in enumerate(target_basis)}
    N = comb(r+1, d+1, exact=True)
    M = comb(r+1, d, exact=True)
    D = scipy.sparse.csr_matrix((M, N), dtype=np.int8)
    for j in range(d+1):
        jth_faces_ix = [
                target_basis_ix[tuple(np.concatenate((l[:j], l[j+1:])))]
                for l in domain_basis
                ]
        D += scipy.sparse.csr_matrix ( 
                        ([(-1)**j]*N, (jth_faces_ix, range(N))), 
                        shape=(M, N), dtype=np.int8
                        )
    return D

def construct_coboundary(d, r=19):
    '''...'''
    return construct_boundary(d+1,r).T

In [4]:
def construct_weights(d, inverse=False, correlation='TC'):
    num_rows = comb(20, d+1, exact=True)
    weights = np.zeros((num_rows, 1), dtype=np.float)
    for p in patients:
        data = import_data(p, d)
        weights += data[correlation]
    weights /= 164
    
    if inverse:
        weights = np.reciprocal(weights)
    
    weights = np.reshape(weights, (num_rows,))
    
    return scipy.sparse.csr_matrix (( 
                         weights, (range(num_rows), range(num_rows))), 
                         shape=(num_rows, num_rows), dtype=float
                         )

## Laplacian

$L^{up}_i = W_i^{-1}\, B_{i+1}\, W_{i+1}\, B_i^T$

$L^{down}_i = B_{i-1}^T\, W_{i-1}^{-1}\, B_i\, W_i$

$L = L^{up} + L^{down}$

In [5]:
print(' d=3', comb(20,4),
      '\n d=4', comb(20,5),
      '\n d=5', comb(20,6) )

 d=3 4845.0 
 d=4 15504.0 
 d=5 38760.0


In [6]:
def construct_up_laplacian(d, inverted_weights=False, correlation='TC'):
    '''Lu = WIa Bp Wp BTa'''
    BTa = construct_coboundary(d)
    Wp = construct_weights(d+1, inverse=False^inverted_weights, correlation='TC')
    Bp = construct_boundary(d+1)
    WIa = construct_weights(d, inverse=True^inverted_weights, correlation='TC')
    Lu = np.multiply(WIa, np.multiply(Bp ,np.multiply(Wp, BTa)))
    
    return Lu

def construct_down_laplacian(d, inverted_weights=False, correlation='TC'):
    '''Ld = BTm WIm Ba Wa'''
    Wa = construct_weights(d, inverse=False^inverted_weights, correlation='TC')
    Ba = construct_boundary(d)
    WIm = construct_weights(d-1, inverse=True^inverted_weights, correlation='TC')
    BTm = construct_coboundary(d-1)
    Ld = np.multiply(BTm, np.multiply(WIm, np.multiply(Ba, Wa)))
    
    return Ld

def construct_laplacian(d, inverted_weights=False, correlation='TC'):
    '''...'''
    Ld = construct_down_laplacian(d, inverted_weights, correlation='TC')
    Lu = construct_up_laplacian(d, inverted_weights, correlation='TC')

    return Ld + Lu

## Main

In [ ]:
patients = [f'00{i}' for i in range(1,10) ] \
         + [f'0{i}' for i in range(10,100)] \
         + [f'{i}' for i in range(100,165)]

for correlation in ['TC', 'DTC']:
    dual = ''
    if correlation == 'DTC':
        dual = '_dual'
    for inverted_weights in [False, True]:
        inv = ''
        if inverted_weights:
            inv += '_inv'
        for d in range(5,6):
            # computing matrix of eigenvectors and storing eigenvalues
            os.mkdir(f'{d}eigendata{dual}{inv}')
            L = construct_laplacian(d, inverted_weights, correlation)
            eigens = np.linalg.eig(L.todense())
            np.save(f'{d}eigendata{dual}{inv}/eigenvalues.npy', eigens[0])
            # decomposing each O-information cochain on the eigenbasis
            for patient in patients:    
                data = import_data(patient, d)
                cochain = data['Oinfo']
                x = np.linalg.solve(eigens[1], cochain)
                np.save(f'{d}eigendata{dual}{inv}/{d}harmonic{patient}.npy', x) #save